## Image classifier (least squares method)

This is simple classifier based on [least squares method](https://en.wikipedia.org/wiki/Least_squares).

First of all, let's import numpy and read data.

In [1]:
import numpy as np

Data is written in two files (<i>mnist_train.csv</i> and <i>mnist_test.csv</i>) picked from [here](https://pjreddie.com/projects/mnist-in-csv/). 

<b>Attention!</b> Reading and parsing big csv files needs some memory (up to a couple of gigabytes) and some calculation power, so it's better to run the following block of code only once.

In [2]:
train_data = np.loadtxt("mnist_train.csv", delimiter=',')
test_data = np.loadtxt("mnist_test.csv", delimiter=',')

As you can see, we have 60k vectors representing images in `train_data` and 10k vectors in `test_data`. Every vector has 785 values - first one is the tag (a digit from 0 to 9) and next 784 numbers represent the image of this digit (actually it is reshaped matrix of pixels 28x28).

In [3]:
print(train_data.shape)
print(test_data.shape)

(60000, 785)
(10000, 785)


So, let's start. Our algorithm has complexity of $O(N \times M \times L)$, where $N$ is the size of train selection, $M$ is the size of test selection and $L$ represents the length of vectors in selections. Therefore, we need to choose  subsets of our selections and  work with it, no with the full data. 

We still can take full data, but then computations will require a lot of time.

In [4]:
train_N = 10000
test_N = 1000
IMAGE_LENGTH = test_data.shape[1] - 1

In [5]:
def get_random_subset(data, new_size):
    old_size = data.shape[0]
    indexes = np.random.choice(old_size, new_size, replace = False)
    labels = data[indexes, 0].astype(int)
    images = data[indexes, 1:]
    return labels, images

train_labels, train_img = get_random_subset(train_data, train_N)
test_labels, test_img = get_random_subset(test_data, test_N)

Now, let's write our classifier itself. As said above, we will use the least squares method. 

How does it work?
For every sample from test selection (which is a vector), we need to find the closest (by Euclidean distance) vector from train selection. The label of the closest vector will be the "predicted" label of vector from test selection.



<b>Comment:</b> our algorithm creates matrix of size $N \times M \times L$, where $N$ is size of train selection, $M$ is size of test selection and $L$ is the length of vectors. So, we need $O(N \times M \times L)$ memory (about $10^4 \cdot 6 \cdot 10^4 \cdot 784 \cdot SizeOfFloat \approx 1,9 \cdot 10^{12}$ bytes of memory for maximal size of train and test data and we cant handle so much). Therefore, it's better to process test data one-by-one. It will rise time usage (not so much, actually), but will reduce memory usage. Space-time trade-off as it is.

In [6]:
def classify_images(train_img, train_labels, test_img, N_train, N_test):
    Tr = train_img.reshape(N_train, 1, IMAGE_LENGTH)
    Te = test_img.reshape(1, N_test, IMAGE_LENGTH)
    DM = np.square(Te - Tr).sum(axis = 2)
    indexes = DM.argmin(axis = 0)
    return train_labels[indexes]

Now let's test our algorithm. We will also measure time of execution using `time` library. 

In [7]:
import time

success_count = 0

begin_time = time.time()

for label, test in zip(test_labels, test_img):
    res = classify_images(train_img, train_labels, test, train_N, 1)
    if(res[0] == label):
        success_count += 1

end_time = time.time()
        
accuracy = success_count / test_N

print("SAMPLES COUNT :", train_N)
print("TESTS COUNT   :", test_N)
print("ELAPSED TIME  :", np.round(end_time - begin_time, 2), "seconds")
print("ACCURACY      :", np.round(accuracy * 100, 2), '%')

SAMPLES COUNT : 10000
TESTS COUNT   : 1000
ELAPSED TIME  : 58.95 seconds
ACCURACY      : 95.2 %


Well done!

You can see how accuracy depends on size of train set by changing `train_N` and restarting code by your own.